In [1]:
from google.colab import files
import zipfile

uploaded = files.upload()

for fn in uploaded.keys():
  with zipfile.ZipFile(fn, 'r') as zip_ref:
    zip_ref.extractall('./')


Saving dataset.zip to dataset.zip


In [2]:
# Libraries
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras import backend as K
from sklearn.model_selection import train_test_split
import numpy as np
import os

In [3]:
# Set path to the dataset
data_path = '/content/Dataset'

In [4]:
# Set image size and batch size
img_width, img_height = 299, 299
batch_size = 32

In [5]:
# Load the InceptionV3 model without the top layers
base_model = InceptionV3(weights='imagenet', include_top=False)

In [6]:
# Add new top layers for the new classification task
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(4, activation='softmax')(x)  # Adjusted for 4 classes
model = Model(inputs=base_model.input, outputs=predictions)

In [7]:
# Freeze the original InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

In [8]:
# Compile the model
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [9]:
# Prepare the data
X = []
Y = []
labels = ['cloudy', 'shine', 'rain', 'sunrise']
for i, label in enumerate(labels):
    folder_path = os.path.join(data_path, label)
    for img_name in os.listdir(folder_path):
        img_path = os.path.join(folder_path, img_name)
        img = image.load_img(img_path, target_size=(img_width, img_height))
        x = image.img_to_array(img)
        X.append(x)
        Y.append(i)
X = np.array(X)
Y = np.array(Y)
Y = np.eye(len(labels))[Y]

In [10]:
# Split the data into training, validation, and test sets
x_train, x_val, y_train, y_val = train_test_split(X, Y, test_size=0.2, random_state=42)
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.1, random_state=42)

In [11]:
# Train the model
history = model.fit(x_train, y_train, batch_size=batch_size, epochs=10, validation_data=(x_val, y_val))

Epoch 1/10
26/26 [==============================] - 282s 11s/step - loss: 65.7299 - accuracy: 0.3432 - val_loss: 22.1156 - val_accuracy: 0.4133
Epoch 2/10
26/26 [==============================] - 253s 10s/step - loss: 15.4704 - accuracy: 0.4741 - val_loss: 9.6509 - val_accuracy: 0.5244
Epoch 3/10
26/26 [==============================] - 275s 11s/step - loss: 10.7144 - accuracy: 0.5160 - val_loss: 4.6963 - val_accuracy: 0.5822
Epoch 4/10
26/26 [==============================] - 275s 11s/step - loss: 8.3261 - accuracy: 0.4988 - val_loss: 7.6516 - val_accuracy: 0.4622
Epoch 5/10
26/26 [==============================] - 246s 10s/step - loss: 6.1365 - accuracy: 0.5346 - val_loss: 16.0467 - val_accuracy: 0.4444
Epoch 6/10
26/26 [==============================] - 273s 11s/step - loss: 5.3117 - accuracy: 0.5642 - val_loss: 7.6914 - val_accuracy: 0.4889
Epoch 7/10
26/26 [==============================] - 246s 10s/step - loss: 4.7955 - accuracy: 0.5333 - val_loss: 6.7659 - val_accuracy: 0.4044
E

In [12]:
# Evaluate the model on the test set
loss, acc = model.evaluate(x_test, y_test, batch_size=batch_size)
print("Test loss:", loss)
print("Test accuracy:", acc)

3/3 [==============================] - 22s 7s/step - loss: 4.6300 - accuracy: 0.4667
Test loss: 4.630026340484619
Test accuracy: 0.46666666865348816
